In [1]:
import time 

import torch
import torch.nn as nn 
import torch.nn.functional as F
import torch.optim as optim 

* ```ResNet``` significantly changed the view of how to parametrize the functions in deep networks.
    * ```DenseNet``` is to some extent the logical extension of this.

* ```Dense_Blocks```
* ```DenseNet``` uses the modified batch normalization, activation, and convolution architecture of ResNet (see the exercise in the previous section).

 Implement this architecture in the ```conv_block``` function.
 
* A ```Dense_Blocks``` consists of multiple ```conv_block``` units, each using the same number of output channels. 

* In the forward computation, however, we concatenate the input and output of each block on the channel dimension.

In [2]:
def conv_block(input_channels, num_channels):
    layers = []
    layers.append(nn.BatchNorm2d(input_channels))
    layers.append(nn.ReLU())
    layers.append(nn.Conv2d(input_channels, num_channels, kernel_size=3, padding=1))
    blk = nn.Sequential(*layers)
    return blk

In [3]:
class DenseBlock(nn.Module):
    def __init__(self, num_convs, input_channels, num_channels, **kwargs):
        super(DenseBlock, self).__init__(**kwargs)
        layer = []
        for i in range(num_convs):
            layer.append(conv_block((num_channels * i + input_channels), num_channels))
        self.net = nn.Sequential(*layer)
    def forward(self, X):
        for blk in self.net:
            Y = blk(X)
            """
            Concatenate the input and output of each block on the channel
            dimension
            """
            X = torch.cat((X, Y), dim=1)
        return X

* we define a convolution block with two blocks of 10 output channels
* When using an input with 3 channels, we will get an output with the 3 + 2 × 10 = 23 channels. 
* The number of convolution block channels controls the increase in the number of output channels relative to the number of input channels.

In [4]:
blk = DenseBlock(2, 3, 10)
X = torch.randn(4, 3, 8, 8)
Y = blk(X)
Y.shape

torch.Size([4, 23, 8, 8])

* ```Transition_Layers```
* Since each dense block will increase the number of channels, adding too many of them will lead to an excessively complex model. 
    * A ```transition layer``` is used to control the complexity of the model.
    * It reduces the number of channels by using the 1 × 1 convolutional layer and halves the height and width of the average pooling layer with a stride of 2, further reducing the complexity of the model.

In [5]:
def transition_block(input_channels, num_channels):
    layers = []
    layers.append(nn.BatchNorm2d(input_channels))
    layers.append(nn.ReLU())
    layers.append(nn.Conv2d(input_channels, num_channels, kernel_size=1))
    layers.append(nn.AvgPool2d(kernel_size=2, stride=2))
    blk = nn.Sequential(*layers)
    return blk

* Apply a transition layer with 10 channels to the output of the dense block in the previous example. 
* This reduces the number of output channels to 10, and halves the height and width.

In [6]:
blk = transition_block(23, 10)
blk(Y).shape

torch.Size([4, 10, 4, 4])

***
* DenseNet Model

In [7]:
class Flatten(torch.nn.Module):
    def forward(self, x):
        return x.view(x.shape[0], -1)

class Reshape(torch.nn.Module):
    def forward(self, x):
        return x.view(-1,1,96,96)

* ```DenseNet``` first uses the same single convolutional layer and maximum pooling layer as ```ResNet```.
    * DenseNet uses four dense blocks. 
* we can set the number of convolutional layers used in each dense block. 
    * Here, we set it to 4,     
* we set the number of channels (i.e. growth rate) for the convolutional layers in the dense block to 32, 
    * so 128 channels will be added to each dense block.

In [8]:
layers = []
layers.append(Reshape())
layers.append(nn.Conv2d(in_channels=1, out_channels=64, kernel_size=7, stride=2, padding=3))
layers.append(nn.BatchNorm2d(64))
layers.append(nn.ReLU())
layers.append(nn.MaxPool2d(kernel_size=3, stride=2, padding=1))

* In ResNet, the height and width are reduced between each module by a residual block with a stride of 2. 
* Here, we use the transition layer to halve the height and width and halve the number of channels.

In [9]:
# Num_channels: the current number of channels
num_channels, growth_rate = 64, 32
num_convs_in_dense_blocks = [4, 4, 4, 4]

for i, num_convs in enumerate(num_convs_in_dense_blocks):
    layers.append(DenseBlock(num_convs, num_channels, growth_rate))
    # This is the number of output channels in the previous dense block
    num_channels += num_convs * growth_rate
    """
    A transition layer that haves the number of channels is added between
    the dense blocks"""
    
    if i != len(num_convs_in_dense_blocks) - 1:
        layers.append(transition_block(num_channels, num_channels // 2))
        num_channels = num_channels // 2

* Similar to ResNet, a global pooling layer and fully connected layer are connected at the end to produce the output.

In [10]:
layers.append(nn.BatchNorm2d(num_channels))
layers.append(nn.ReLU())
layers.append(nn.AdaptiveMaxPool2d((1,1)))
layers.append(Flatten())
layers.append(nn.Linear(num_channels, 10))

net = nn.Sequential(*layers)

***
* Check the model design 
* To have a reasonable training time on Fashion-MNIST, we reduce the input height and width from 224 to 96

In [11]:
X = torch.rand(size=(1, 1, 96, 96))   # [B, C, H, W]

for layer in net:
    X = layer(X)
    print(layer.__class__.__name__,'output shape:\t', X.shape)

Reshape output shape:	 torch.Size([1, 1, 96, 96])
Conv2d output shape:	 torch.Size([1, 64, 48, 48])
BatchNorm2d output shape:	 torch.Size([1, 64, 48, 48])
ReLU output shape:	 torch.Size([1, 64, 48, 48])
MaxPool2d output shape:	 torch.Size([1, 64, 24, 24])
DenseBlock output shape:	 torch.Size([1, 192, 24, 24])
Sequential output shape:	 torch.Size([1, 96, 12, 12])
DenseBlock output shape:	 torch.Size([1, 224, 12, 12])
Sequential output shape:	 torch.Size([1, 112, 6, 6])
DenseBlock output shape:	 torch.Size([1, 240, 6, 6])
Sequential output shape:	 torch.Size([1, 120, 3, 3])
DenseBlock output shape:	 torch.Size([1, 248, 3, 3])
BatchNorm2d output shape:	 torch.Size([1, 248, 3, 3])
ReLU output shape:	 torch.Size([1, 248, 3, 3])
AdaptiveMaxPool2d output shape:	 torch.Size([1, 248, 1, 1])
Flatten output shape:	 torch.Size([1, 248])
Linear output shape:	 torch.Size([1, 10])


*** 
* Data Acquisition 
* Reading Data (Fashion-MNIST)
* Preprocess: Fashion-MNIST has 28x28 pixels -> upsample them to 96x96

In [12]:
import sys 
import os 

import torchvision 
from torchvision import transforms 
from torch.utils.data import DataLoader 

def load_data_fashion_mnist(batch_size, resize=None, root=os.path.join(os.getcwd(), 'datasets', 'fashion-mnist')):
    """Download the Fashion-MNIST dataset and then load into memory."""
    root = os.path.expanduser(root)
    transformer = []
    if resize:
        transformer += [transforms.Resize(resize)]
    transformer += [transforms.ToTensor()]
    transformer = transforms.Compose(transformer)

    mnist_train = torchvision.datasets.FashionMNIST(root=root, train=True, transform=transformer, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(root=root, train=False, transform=transformer, download=True)
    num_workers = 0 if sys.platform.startswith('win32') else 4

    train_iter = DataLoader(mnist_train, batch_size, shuffle=True, num_workers=num_workers)
    test_iter = DataLoader(mnist_test, batch_size, shuffle=False, num_workers=num_workers)
    return train_iter, test_iter

In [13]:
batch_size = 128

#Loading Fashion-MNIST Dataset
train_iter, test_iter = load_data_fashion_mnist(batch_size, resize=96)

***
* Model Training

In [14]:
def try_gpu():
    """If GPU is available, return torch.device as cuda:0; else return torch.device as cpu."""
    if torch.cuda.is_available():
        device = torch.device('cuda:0')
    else:
        device = torch.device('cpu')
    return device

In [15]:
def evaluate_accuracy(data_iter, net, device=torch.device('cpu')):
    """Evaluate accuracy of a model on the given data set."""
    net.eval()  # Switch to evaluation mode for Dropout, BatchNorm etc layers.
    acc_sum, n = torch.tensor([0], dtype=torch.float32, device=device), 0
    for X, y in data_iter:
        # Copy the data to device.
        X, y = X.to(device), y.to(device)
        with torch.no_grad():
            y = y.long()
            acc_sum += torch.sum((torch.argmax(net(X), dim=1) == y))
            n += y.shape[0]
    return acc_sum.item()/n

In [16]:
def train_ch5(net, train_iter, test_iter, criterion, num_epochs, batch_size, device, lr=None):
    """Train and evaluate a model with CPU or GPU."""
    print('training on', device)
    net.to(device)
    optimizer = optim.SGD(net.parameters(), lr=lr)
    for epoch in range(num_epochs):
        net.train() # Switch to training mode
        n, start = 0, time.time()
        train_l_sum = torch.tensor([0.0], dtype=torch.float32, device=device)
        train_acc_sum = torch.tensor([0.0], dtype=torch.float32, device=device)
        for X, y in train_iter:
            optimizer.zero_grad()
            X, y = X.to(device), y.to(device) 
            y_hat = net(X)
            loss = criterion(y_hat, y)
            loss.backward()
            optimizer.step()
            with torch.no_grad():
                y = y.long()
                train_l_sum += loss.float()
                train_acc_sum += (torch.sum((torch.argmax(y_hat, dim=1) == y))).float()
                n += y.shape[0]

        test_acc = evaluate_accuracy(test_iter, net, device) 
        print('epoch %d, loss %.4f, train acc %.3f, test acc %.3f, time %.1f sec'\
            % (epoch + 1, train_l_sum/n, train_acc_sum/n, test_acc, time.time() - start))

In [17]:
lr, num_epochs, batch_size, device =  0.05, 5, 256, try_gpu()

#Xavier initialization of weights
def init_weights(m):
    if type(m) == nn.Linear or type(m) == nn.Conv2d:
        torch.nn.init.xavier_uniform_(m.weight)

In [18]:
net.apply(init_weights)
net = net.to(device)

In [19]:
#Loss Function Criterion
criterion = nn.CrossEntropyLoss()

train_ch5(net, train_iter, test_iter, criterion, num_epochs, batch_size, device, lr)

training on cuda:0
epoch 1, loss 0.0041, train acc 0.815, test acc 0.865, time 20.1 sec
epoch 2, loss 0.0024, train acc 0.887, test acc 0.884, time 20.0 sec
epoch 3, loss 0.0020, train acc 0.904, test acc 0.897, time 20.0 sec
epoch 4, loss 0.0018, train acc 0.915, test acc 0.800, time 20.2 sec
epoch 5, loss 0.0016, train acc 0.923, test acc 0.842, time 20.3 sec
